In [1]:
import sys
sys.path.append('/home/nick/projects')
import importlib
from src.PointGPT_utils.models import pt
import torch
from src.PointGPT_utils.dataset import FinDataset, PartNormalDataset
import numpy as np

/home/nick/miniconda3/envs/point-gpt-env/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [2]:
model_name = 'PointGPT_S'
MODEL = pt
num_part = 50
npoint = 2048

def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    new_y = torch.eye(num_classes)[y.cpu().data.numpy(),]
    if (y.is_cuda):
        return new_y.cuda()
    return new_y

def inplace_relu(m):
    classname = m.__class__.__name__
    if classname.find('ReLU') != -1:
        m.inplace = True

In [3]:
if model_name == 'PointGPT_S':
    classifier = MODEL.get_model(num_part, trans_dim=384, depth=12, drop_path_rate=0.1, num_heads=6, decoder_depth=4, group_size=32, num_group=128, prop_dim=1024, label_dim1=512, label_dim2=256, encoder_dims=384)
    classifier = classifier.cuda()
    criterion = MODEL.get_loss().cuda()
    classifier.apply(inplace_relu)


# try loading from a checkpoint
# ckpt_path = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/PointGPT/pretrained.pth"
ckpt_path = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/PointGPT/part_seg.pth"
mdl_ckpt = torch.load(ckpt_path)
classifier.load_state_dict(mdl_ckpt["model_state_dict"])
# classifier.load_model_from_ckpt(ckpt_path)

<All keys matched successfully>

In [4]:
root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/PointGPT/data/ShapeNet55/"

In [5]:
# try loading pointcloud data
# root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/PointGPT/data/ShapeNet55/"
root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/point_cloud_data/nucleus_point_clouds/"
TRAIN_DATASET = FinDataset(
        root=root, npoints=npoint, split='train')

trainDataLoader = torch.utils.data.DataLoader(TRAIN_DATASET, batch_size=16, shuffle=True)

test = next(iter(trainDataLoader))

points = test[0]
label = test[1]
target = test[2]

points, label, target = points.float().cuda(), label.long().cuda(), target.long().cuda()
points = points.transpose(2, 1)

In [6]:
# point_features = classifier.forward(points, label)
local_features, glb0, glb1, glb2 = classifier.extract_features(points, label)
glb1.shape

torch.Size([16, 256, 2048])

In [31]:
import plotly.express as px
import numpy as np
ind = 6
points0 = np.squeeze(points[ind].cpu().numpy()).T
point0_features = np.squeeze(glb0[ind].detach().cpu().numpy()).T

fig = px.scatter_3d(x=points0[:, 0], y=points0[:, 1], z=points0[:, 2], color=point0_features[:, 120])
fig.show()

In [32]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(point0_features)
pca_array = pca.transform(point0_features)
pca_array.shape

(2048, 3)

In [35]:
fig = px.scatter_3d(x=points0[:, 0], y=points0[:, 1], z=points0[:, 2], color=pca_array[:, 1])
fig.show()

In [36]:
import pandas as pd
test_path = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/point_cloud_data/nucleus_point_features/20240619_well0000_time0044_points_features.csv"
test_points = pd.read_csv(test_path)
print(test_points.shape)

(7690, 522)


In [38]:
import plotly.express as px
fig = px.scatter_3d(test_points, x="X", y="Y", z="Z", color="feat_0006")
fig.show()

In [39]:
from sklearn.decomposition import PCA

feature_cols = [col for col in test_points.columns if "feat_" in col]
feature_array = test_points.loc[:, feature_cols].to_numpy()

pca = PCA(n_components=3)
pca.fit(feature_array)
pca_array = pca.transform(feature_array)
pca_array.shape

(7690, 3)

In [42]:
fig = px.scatter_3d(test_points, x="X", y="Y", z="Z", color=pca_array[:, 2])
fig.show()

In [29]:
test_points.shape

(7836, 522)